## 1、晶胞参数相关计算

- 1.1 晶面间距计算
- 1.2 晶带轴、晶面夹角计算
- 1.3 晶向和晶面的转换
- 1.4 叉乘计算（两晶面所在晶带轴、两晶带轴所在晶面）
- 1.5晶面筛选  
- 1.6 晶向筛选（批量计算）

#### 晶胞参数输入(nm)：（手动输入或者直接读取，二者选其一即可）

In [1]:
# 方式1：手动输入晶胞参数

a = 0.5828
b = 0.5828
c = 0.5828
alpha = 90
beta = 90
gamma = 90

In [1]:
# 方式2：直接通过cif或者POSCAR文件读取晶胞参数
# 需要ase和tkinter库

from ase.io import read
from tkinter import Tk, filedialog

# 弹窗选择文件
root = Tk()
root.withdraw()  # 隐藏主窗口
file_path = filedialog.askopenfilename(
    title='Choose CIF or POSCAR file',
    filetypes=[("Structure files", "*.cif *.vasp *.poscar *.CONTCAR *.POSCAR"), ("All files", "*.*")]
)

if not file_path:
    print("No file selected.")
    exit()

# 读取结构
atoms = read(file_path)

# 提取晶胞参数（a, b, c, alpha, beta, gamma）
cell_lengths = atoms.get_cell_lengths_and_angles()

a, b, c, alpha, beta, gamma = cell_lengths

print("=== Cell Parameters ===")
print(f"a     = {a:.6f} Å")
print(f"b     = {b:.6f} Å")
print(f"c     = {c:.6f} Å")
print(f"alpha = {alpha:.6f} °")
print(f"beta  = {beta:.6f} °")
print(f"gamma = {gamma:.6f} °")

1
=== Cell Parameters ===
a     = 8.370000 Å
b     = 8.370000 Å
c     = 7.290000 Å
alpha = 90.000000 °
beta  = 90.000000 °
gamma = 120.000000 °


C:\Users\shiki\AppData\Local\Temp\ipykernel_26916\3683799412.py:20: DeprecationWarning: Please use atoms.cell.cellpar() instead
  cell_lengths = atoms.get_cell_lengths_and_angles()


In [3]:
import numpy as np

class Lattice:
    def __init__(self, a, b, c, alpha, beta, gamma):
        """
        Initialize lattice parameters.

        Parameters:
            a, b, c : lattice constants (Å)
            alpha, beta, gamma : lattice angles (degrees)
        """

        self.a = a
        self.b = b
        self.c = c
        self.alpha = alpha
        self.beta = beta
        self.gamma = gamma

        # Build metric tensors
        self.G = self._build_G()
        self.G_star = np.linalg.inv(self.G)

    def _build_G(self):
        """Construct the real-space metric tensor G (3×3)."""

        α = np.deg2rad(self.alpha)
        β = np.deg2rad(self.beta)
        γ = np.deg2rad(self.gamma)

        G = np.array([
            [self.a*self.a, self.a*self.b*np.cos(γ), self.a*self.c*np.cos(β)],
            [self.a*self.b*np.cos(γ), self.b*self.b, self.b*self.c*np.cos(α)],
            [self.a*self.c*np.cos(β), self.b*self.c*np.cos(α), self.c*self.c]
        ])

        return G

    def d_hkl(self, h, k, l):
        """
        Compute d-spacing of plane (h k l):

            d = 1 / sqrt(hkl^T · G* · hkl)
        """

        hkl = np.array([h, k, l], dtype=float)
        inv_d2 = np.dot(hkl, np.dot(self.G_star, hkl))
        d = 1.0 / np.sqrt(inv_d2)
        return d
        
    def plane_angle(self, hkl1, hkl2):
        """
        Compute the angle (degrees) between two planes.

        Formula:
            cosθ = (h1·G*·h2) / (|h1|·|h2|)
        """

        h1 = np.array(hkl1, dtype=float)
        h2 = np.array(hkl2, dtype=float)
        cos_theta = np.dot(h1, np.dot(self.G_star, h2)) / \
                    (np.sqrt(np.dot(h1, np.dot(self.G_star, h1))) *
                     np.sqrt(np.dot(h2, np.dot(self.G_star, h2))))
        return np.degrees(np.arccos(cos_theta))

    def direction_angle(self, uvw1, uvw2):
        """
        Compute angle (degrees) between directions [uvw].

        Formula:
            cosθ = (v1·G·v2) / (|v1|·|v2|)
        """

        v1 = np.array(uvw1, dtype=float)
        v2 = np.array(uvw2, dtype=float)
        cos_theta = np.dot(v1, np.dot(self.G, v2)) / \
                    (np.sqrt(np.dot(v1, np.dot(self.G, v1))) *
                     np.sqrt(np.dot(v2, np.dot(self.G, v2))))
        return np.degrees(np.arccos(cos_theta))
    
    # -------- Direction ↔ Plane --------
    def hkl_from_dir(self, uvw, tol=0.05):
        """
        Convert direction [uvw] to plane (hkl).

        Returns:
            hkl_float : float-valued hkl
            hkl_int   : integer hkl ratio
        """
        u = np.array(uvw, dtype=float)
        hkl_float = np.dot(self.G, u)
        hkl_int = self._to_integer_ratio(hkl_float, tol)
        return hkl_float, hkl_int

    def dir_from_hkl(self, hkl, tol=0.05):
        """
        Convert plane (hkl) to direction [uvw].

        Returns:
            uvw_float : float-valued direction
            uvw_int   : integer direction ratio
        """
        h = np.array(hkl, dtype=float)
        uvw_float = np.dot(np.linalg.inv(self.G), h)
        uvw_int = self._to_integer_ratio(uvw_float, tol)
        return uvw_float, uvw_int

    # ------------------- Utility Functions -------------------
    def _to_integer_ratio(self, vec, tol=0.05):
        """
        Convert a vector to smallest integer ratio within tolerance.

        Parameters:
            vec : input vector
            tol : allowed relative deviation
        """

        vec = np.array(vec, dtype=float)
        non_zero = vec[np.abs(vec) > 1e-8]
        if len(non_zero) == 0:
            return np.zeros_like(vec, dtype=int)
    
        scale = np.min(np.abs(non_zero))
        vec_scaled = vec / scale
    
        multiplier = 1
        max_iter = 100
        for _ in range(max_iter):
            vec_rounded = np.round(vec_scaled * multiplier)
            error = np.abs(vec_scaled * multiplier - vec_rounded)
            if np.all(error < tol):
                break
            multiplier += 1
    
        vec_int = np.round(vec_scaled * multiplier).astype(int)
        return vec_int

    def direction_from_planes(self, hkl1, hkl2, tol=0.05):
        """
        Compute a zone axis [uvw] perpendicular to two planes.
        
        Returns:
            uvw_float, uvw_int
        """

        h1 = np.array(hkl1, dtype=float)
        h2 = np.array(hkl2, dtype=float)
    
        # Cross product gives the zone axis
        u_float = np.cross(h1, h2)
        u_int = self._to_integer_ratio(u_float, tol)
        return u_float, u_int
        
    def plane_from_directions(self, uvw1, uvw2, tol=0.05):
        """
        Compute a plane (hkl) perpendicular to two crystal directions
        by converting directions -> hkl_float, crossing them, then converting
        the resulting direction back to hkl via hkl_from_dir.

        Returns:
            hkl_float, hkl_int
        """
        # 1) direction -> hkl_float using existing method
        h1_float, h1_int = self.hkl_from_dir(uvw1, tol=tol)
        h2_float, h2_int = self.hkl_from_dir(uvw2, tol=tol)

        # print(h1_float, h2_float)

        # 2) cross product of the two hkl_float vectors (gives a direction in that space)
        n_dir = np.cross(h1_float, h2_float)

        # print(n_dir)

        # handle degenerate case (parallel or zero)
        if np.all(np.abs(n_dir) < 1e-12):
            # No unique plane (directions may be parallel or produce zero cross)
            return np.array([0.0, 0.0, 0.0]), np.array([0, 0, 0], dtype=int)

        # 3) convert the direction n_dir back to hkl using hkl_from_dir
        #    (hkl_from_dir returns (hkl_float, hkl_int))
        hkl_float, hkl_int = self.hkl_from_dir(n_dir, tol=tol)

        return hkl_float, hkl_int

    def list_planes_by_dspacing(self, max_index=5, d_min=0.1, ref_hkl=None,
                                angle_filter=None, tol_angle=1.0):
        """
        List all planes (hkl) with d-spacing > d_min.
        Optionally compute angle to a reference plane and filter by angle.

        Parameters:
            max_index   : search range for Miller indices
            d_min       : minimum d-spacing
            ref_hkl     : reference plane (optional)
            angle_filter: target angle to filter (optional)
            tol_angle   : angle tolerance (degrees)

        Returns:
            List of tuples: (h, k, l, d_hkl, angle)
            Sorted by d_hkl descending.
        """

        planes = []
        for h in range(-max_index, max_index+1):
            for k in range(-max_index, max_index+1):
                for l in range(-max_index, max_index+1):
                    if h == 0 and k == 0 and l == 0:
                        continue

                    d = self.d_hkl(h, k, l)
                    if d < d_min:
                        continue
    
                    angle_deg = None
                    if ref_hkl is not None:
                        angle_deg = self.plane_angle((h, k, l), ref_hkl)
                    
                    if angle_filter is not None and angle_deg is not None:
                        if abs(angle_deg - angle_filter) > tol_angle:
                            continue
    
                    planes.append((h, k, l, d, angle_deg))
        
        return sorted(planes, key=lambda x: x[3], reverse=True)
        
    def sort_dir(self, max_index, ref_uvw, angle_filter, tol_angle=1.0):
        """
        List all unique crystal directions [uvw] whose angle to a reference
        direction is within a given range. Directions equivalent by scaling
        (±k·[uvw]) are removed.

        Parameters:
            max_index   : max absolute index for u,v,w search range
            ref_uvw     : reference direction [u v w]
            angle_filter: target angle (deg)
            tol_angle   : allowed deviation (deg)

        Returns:
            List of tuples: (u, v, w, angle_deg)
            Sorted by angle.
        """

        ref = np.array(ref_uvw, dtype=float)
        dirs = []
        unique_set = set()  # store normalized unique directions

        for u in range(-max_index, max_index + 1):
            for v in range(-max_index, max_index + 1):
                for w in range(-max_index, max_index + 1):

                    if u == 0 and v == 0 and w == 0:
                        continue

                    uvw = np.array([u, v, w], dtype=float)
                    angle = self.direction_angle(uvw, ref)

                    if abs(angle - angle_filter) > tol_angle:
                        continue

                    # ---- Normalize direction to unique form ----
                    # 1. gcd normalization
                    g = np.gcd.reduce([int(abs(u)), int(abs(v)), int(abs(w))])
                    if g != 0:
                        nu, nv, nw = u // g, v // g, w // g
                    else:
                        nu, nv, nw = u, v, w

                    # 2. enforce sign convention: first non-zero > 0
                    if nu < 0 or (nu == 0 and nv < 0) or (nu == 0 and nv == 0 and nw < 0):
                        nu, nv, nw = -nu, -nv, -nw

                    key = (nu, nv, nw)
                    if key in unique_set:
                        continue
                    unique_set.add(key)

                    dirs.append((nu, nv, nw, angle))

        dirs_sorted = sorted(dirs, key=lambda x: x[3])
        return dirs_sorted

# Create lattice object
lattice = Lattice(a, b, c, alpha, beta, gamma)

#### 输出:G矩阵

In [4]:
G_matrix = lattice.G
G_star_matrix = lattice.G_star

# 输出
print("G =", G_matrix)
print("G* =", G_star_matrix)

G = [[ 7.00568981e+01 -3.50284490e+01  3.73623199e-15]
 [-3.50284490e+01  7.00568981e+01  3.73623199e-15]
 [ 3.73623199e-15  3.73623199e-15  5.31440994e+01]]
G* = [[ 1.90321491e-02  9.51607457e-03 -2.00704853e-18]
 [ 9.51607457e-03  1.90321491e-02 -2.00704853e-18]
 [-2.00704853e-18 -2.00704853e-18  1.88167644e-02]]


#### 1.1 晶面间距计算

In [5]:
# 输入需要计算晶面间距的晶面（用逗号隔开）
h, k, l = 5, 5, 6

# 输出晶面间距
d_hkl = lattice.d_hkl(h, k, l)
print("d(111) =", d_hkl)

d(111) = 0.6892758544971332


#### 1.2 晶带轴、晶面夹角计算

In [198]:
# 输入两个晶面
h1, k1, l1 = 1, 0, 1
h2, k2, l2 = 1, 1, 1

# 计算两个晶面夹角
angle_planes = lattice.plane_angle([h1,k1,l1], [h2,k2,l2])
print("Angle between (h1, k1, l1) and (h2, k2, l2 ) planes =", angle_planes)

Angle between (h1, k1, l1) and (h2, k2, l2 ) planes = 28.653247347572716


In [199]:
# 输入两个晶向
u1, v1, w1 = -1, 0, 0
u2, v2, w2 = -9, 1, 0

# 计算两个晶带夹角
angle_dirs = lattice.direction_angle([u1, v1, w1], [u2, v2, w2])
print("Angle between [u1, v1, w1] and [u2, v2, w2] directions =", angle_dirs)

Angle between [u1, v1, w1] and [u2, v2, w2] directions = 10.907837816629748


#### 1.3 晶向和晶面的转换

In [200]:
# 晶向对应的晶面法线
u, v, w = -1, 0, 0
# 转化为整数允许的误差（一般0.05 ~ 0.5 * 100%）
tol = 0.06

uvw = [u, v, w]
hkl_float, hkl_int  = lattice.hkl_from_dir(uvw, tol)
print(f"[{uvw}] → (hkl) plane: float = {hkl_float}, int = {hkl_int}")

[[-1, 0, 0]] → (hkl) plane: float = [-5.09496164e-01 -5.41093055e-17  1.46928982e-01], int = [-52   0  15]


In [201]:
# 晶面法线对应的晶向
h, k, l = -0.5, -0.5, 1
# 转化为整数允许的误差（一般0.05 ~ 0.5 * 100%）
tol = 0.1

hkl = [h, k, l]
uvw_float, uvw_int = lattice.dir_from_hkl(hkl, tol)
print(f"(hkl) {hkl} → [uvw] direction: float = {uvw_float}, int = {uvw_int}")

(hkl) [-0.5, -0.5, 1] → [uvw] direction: float = [-0.94827755 -0.32623362  0.11472363], int = [-157  -54   19]


#### 1.4 叉乘计算（两晶面所在晶带轴、两晶带轴所在晶面）

In [237]:
# 输入两个晶面
h1, k1, l1 = -1, 1, 3
h2, k2, l2 = 5, -1, 5
# 转化为整数允许的误差（一般0.05 ~ 0.5 * 100%）
tol = 0.2

hkl1 = [h1, k1, l1]
hkl2 = [h2, k2, l2]
uvw_float, uvw_int = lattice.direction_from_planes(hkl1, hkl2, tol)
print(f"Direction perpendicular to planes {hkl1} & {hkl2}: float = {uvw_float}, int = {uvw_int}")

Direction perpendicular to planes [-1, 1, 3] & [5, -1, 5]: float = [ 8. 20. -4.], int = [ 2  5 -1]


In [236]:
# 输入两个晶向
u1, v1, w1 = -1, 1, 3
u2, v2, w2 = 5, -1, 5
# 转化为整数允许的误差（一般0.05 ~ 0.5 * 100%）
tol = 0.2

uvw1 = [u1, v1, w1]
uvw2 = [u2, v2, w2]
hkl_float, hkl_int = lattice.plane_from_directions(uvw1, uvw2, tol)
print(f"Plane perpendicular to directions {uvw1} & {uvw2}: float = {hkl_float}, int = {hkl_int}")

[-0.33965584  0.33965584  1.01896752] [ 1.6982792  -0.33965584  1.6982792 ]
[ 0.92292872  2.30732179 -0.46146436]
Plane perpendicular to directions [-1, 1, 3] & [5, -1, 5]: float = [ 0.31347813  0.78369532 -0.15673906], int = [ 2  5 -1]


#### 1.5 晶面筛选

In [204]:
# （必选）遍历所有(x,x,x)范围内的晶面间距小于d_min的晶面
max_index = 10
d_min = 0.05

#（可选，不影响输出）输入一个参考晶面，可以计算遍历的晶面和该晶面的夹角
h, k, l = -0.5, -0.5, 1

# （可选，影响输出）输入一个角度，筛选和前面参考晶面夹角为该角度（允许误差：tol_angel）的所有晶面，如果不需要筛选，则设置为 None
angle_filter = 84.95
tol_angle = 0.1


ref_hkl = (h, k, l)
# 输出 d > d_min nm 且与 (h, k, l) 夹角约 angle_filter ± tol_angle
planes = lattice.list_planes_by_dspacing(max_index, d_min, ref_hkl, angle_filter, tol_angle)

for hkl in planes:
    print(f"(hkl) = ({hkl[0]},{hkl[1]},{hkl[2]}),   d = {hkl[3]:.3f} nm,   angle = {hkl[4]:.2f}°")

(hkl) = (-1,-1,2),   d = 0.577 nm,   angle = nan°
(hkl) = (1,1,-2),   d = 0.577 nm,   angle = nan°
(hkl) = (-2,-2,4),   d = 0.288 nm,   angle = nan°
(hkl) = (2,2,-4),   d = 0.288 nm,   angle = nan°
(hkl) = (1,-5,-3),   d = 0.228 nm,   angle = 84.89°
(hkl) = (-2,5,1),   d = 0.203 nm,   angle = 84.89°
(hkl) = (2,-5,6),   d = 0.183 nm,   angle = 84.89°
(hkl) = (-2,7,8),   d = 0.146 nm,   angle = 84.89°
(hkl) = (-4,-4,8),   d = 0.144 nm,   angle = nan°
(hkl) = (4,4,-8),   d = 0.144 nm,   angle = nan°
(hkl) = (3,-9,5),   d = 0.115 nm,   angle = 84.92°
(hkl) = (-5,-5,10),   d = 0.115 nm,   angle = nan°
(hkl) = (5,5,-10),   d = 0.115 nm,   angle = nan°
(hkl) = (2,-10,-6),   d = 0.114 nm,   angle = 84.89°
(hkl) = (-3,10,10),   d = 0.103 nm,   angle = 85.02°
(hkl) = (-4,10,2),   d = 0.102 nm,   angle = 84.89°


C:\Users\shiki\AppData\Local\Temp\ipykernel_4148\4143829678.py:62: RuntimeWarning: invalid value encountered in arccos
  return np.degrees(np.arccos(cos_theta))


#### 1.6 晶向筛选

In [279]:
# 遍历所有[x,x,x]范围内的和参考晶向夹角在 angle_filter ± tol_angle 范围内的所有晶面
max_index = 15
u, v, w = -3, 1, 0
angle_filter = 37
tol_angle = 3

ref_uvw = (u, v, w)
directions = lattice.sort_dir(max_index, ref_uvw, angle_filter, tol_angle)

for u, v, w, angle in directions:
    print(f"[{u} {v} {w}]   angle = {angle:.2f}°")

[13 0 1]   angle = 34.01°
[13 -14 -2]   angle = 34.05°
[15 -4 -3]   angle = 34.08°
[13 -6 -3]   angle = 34.12°
[13 -10 -3]   angle = 34.16°
[12 0 -1]   angle = 34.25°
[12 -11 2]   angle = 34.25°
[5 -6 0]   angle = 34.30°
[15 -11 3]   angle = 34.30°
[7 -7 1]   angle = 34.31°
[10 -11 1]   angle = 34.34°
[11 -13 -1]   angle = 34.34°
[12 -13 -2]   angle = 34.57°
[14 -13 -3]   angle = 34.60°
[15 -1 -2]   angle = 34.62°
[12 0 1]   angle = 34.62°
[9 -8 -2]   angle = 34.63°
[13 -15 1]   angle = 34.65°
[9 -11 0]   angle = 34.71°
[15 -5 3]   angle = 34.72°
[14 -8 3]   angle = 34.74°
[5 -5 -1]   angle = 34.75°
[13 -11 -3]   angle = 34.75°
[14 -7 3]   angle = 34.82°
[13 -13 2]   angle = 34.85°
[9 -10 1]   angle = 34.91°
[6 -1 1]   angle = 34.93°
[14 -9 3]   angle = 34.96°
[5 -4 1]   angle = 35.00°
[9 -3 -2]   angle = 35.10°
[13 -5 -3]   angle = 35.12°
[12 -8 -3]   angle = 35.21°
[13 -15 -2]   angle = 35.22°
[12 -7 -3]   angle = 35.28°
[14 -6 3]   angle = 35.29°
[14 -4 -3]   angle = 35.30°
[9 -11 -

C:\Users\shiki\AppData\Local\Temp\ipykernel_4148\3843201293.py:77: RuntimeWarning: invalid value encountered in arccos
  return np.degrees(np.arccos(cos_theta))


## 2、晶胞转换相关计算

- 2.1 已知转换矩阵求不同坐标系下晶面/晶向的对应关系
- 2.2 已知晶面或者晶向的对应关系求转换矩阵
- 2.3 三个坐标系之间的转化矩阵

In [2]:
import numpy as np

class CrystalTransform:
    """
    Universal class for crystal plane and direction transformations.
    
    Features:
    - Compute transformation matrix S from three pairs of planes or directions.
    - Transform planes (hkl) or directions [uvw] between two bases.
    - Inverse transformations also supported.
    """
    def __init__(self, S=None):
        """
        Initialize with optional transformation matrix S.
        """
        if S is not None:
            self.S = np.array(S)
        else:
            self.S = None

    @staticmethod
    def compute_S_from_planes(h_list, H_list):
        """
        Compute transformation matrix S from three plane pairs H = S @ h.
        """
        h_mat = np.array(h_list).T
        H_mat = np.array(H_list).T
        if np.linalg.det(h_mat) == 0:
            raise ValueError("Original planes are linearly dependent, cannot compute S")
        S = H_mat @ np.linalg.inv(h_mat)
        return S

    @staticmethod
    def compute_S_from_directions(u_list, U_list):
        """
        Compute transformation matrix S from three direction pairs U = (S.T)^(-1) @ u
        """
        u_mat = np.array(u_list).T
        U_mat = np.array(U_list).T
        if np.linalg.det(U_mat) == 0:
            raise ValueError("New directions are linearly dependent, cannot compute S")
        S = (u_mat @ np.linalg.inv(U_mat)).T
        return S

    @staticmethod
    def compute_between_bases_1(S_Aa, S_Aa_prime):
        """
        Compute transformation matrix between two bases a -> a', 
        given their transformations from the same reference A.

        S_Aa: matrix from A -> a
        S_Aa_prime: matrix from A -> a'
        Returns: S_aa' (matrix from a -> a')
        """
        S_aa_prime = S_Aa_prime @ np.linalg.inv(S_Aa)
        
        return S_aa_prime

    @staticmethod
    def compute_between_bases_2(S_Aa, S_aa_prime):
        """
        Compute transformation matrix between two bases A -> a', 
        given transformations from the from A to a and a to a'.

        S_Aa: matrix from A -> a
        S_aa_prime: matrix from a -> a'
        Returns: S_Aa' (matrix from A -> a')
        """
        S_aa_prime = S_aa_prime @ S_Aa
        
        return S_aa_prime

    # ==========================
    # Utility function
    # ==========================
    @staticmethod
    def _to_integer_ratio(vec, tol=0.05):
        vec = np.array(vec, dtype=float)
        non_zero = vec[np.abs(vec) > 1e-8]
        if len(non_zero) == 0:
            return np.zeros_like(vec, dtype=int)

        scale = np.min(np.abs(non_zero))
        vec_scaled = vec / scale

        multiplier = 1
        max_iter = 100
        for _ in range(max_iter):
            vec_rounded = np.round(vec_scaled * multiplier)
            error = np.abs(vec_scaled * multiplier - vec_rounded)
            if np.all(error < tol):
                break
            multiplier += 1

        vec_int = np.round(vec_scaled * multiplier).astype(int)
        return vec_int

    # ==========================
    # Plane transformations
    # ==========================
    def transform_plane(self, h):
        h = np.array(h).reshape(3,1)
        H = self.S @ h
        H_int = self._to_integer_ratio(H.flatten())
        return H.flatten(), H_int

    def inverse_plane(self, H):
        H = np.array(H).reshape(3,1)
        h = np.linalg.inv(self.S) @ H
        h_int = self._to_integer_ratio(h.flatten())
        return h.flatten(), h_int

    # ==========================
    # Direction transformations
    # ==========================
    def transform_direction(self, u):
        u = np.array(u).reshape(3,1)
        U = np.linalg.inv(self.S.T) @ u
        U_int = self._to_integer_ratio(U.flatten())
        return U.flatten(), U_int

    def inverse_direction(self, U):
        U = np.array(U).reshape(3,1)
        u = self.S.T @ U
        u_int = self._to_integer_ratio(u.flatten())
        return u.flatten(), u_int

### 2.1 已知S求晶面/晶向对应关系

In [3]:
# 输入转换矩阵
s11, s12, s13 = -0.5, -0.5, 1
s21, s22, s23 = 0.5, -1, 0.5
s31, s32, s33 = 2.5, 2.5, 3

S = [[s11, s12, s13],
    [s21, s22, s23],
    [s31, s32, s33]]

# 创建对象
ct = CrystalTransform(S)

In [4]:
# 输入原晶面 h
h, k, l = 0, 0, 8

# 计算新的晶面
H_float, H_int = ct.transform_plane([h,k,l])
print("hkl -> HKL (float):", H_float)
print("hkl -> HKL (int):", H_int)

hkl -> HKL (float): [ 8.  4. 24.]
hkl -> HKL (int): [2 1 6]


In [323]:
# 输入原晶向
u, v, w = -1, 1, 0

# 计算新的晶向
U_float, U_int = ct.transform_direction([u, v, w])
print("uvw -> UVW (float):", U_float)
print("uvw -> UVW (int):", U_int)

uvw -> UVW (float): [-0.5  0.5  0. ]
uvw -> UVW (int): [-1  1  0]


In [294]:
# 反向变换晶面（已知转换后的晶面求转换前的晶面）
H, K, L = 1, 3, 10

h_float, h_int = ct.inverse_plane([H, K, L])
print("HKL -> hkl (float):", h_float)
print("HKL -> hkl (int):", h_int)

HKL -> hkl (float): [ 2.54166667 -0.79166667  1.875     ]
HKL -> hkl (int): [ 61 -19  45]


In [325]:
# 反向变换晶向（已知转换后的晶向求转换前的晶向）
U, V, W = -1, 1, 0
u_float, u_int = ct.inverse_direction([U, V, W])
print("UVW -> uvw (float):", u_float)
print("UVW -> uvw (int):", u_int)

UVW -> uvw (float): [ 1.  -0.5 -0.5]
UVW -> uvw (int): [ 2 -1 -1]


### 2.2 已知晶面/晶向对应关系求解S

In [301]:
# ===== 1. 已知三组晶面对应关系，求 S =====
# 输入三组原晶面
h1, k1, l1 = 1, 1, 1
h2, k2, l2 = -1, 1, 1
h3, k3, l3 = 1, -1, 1

# 输入三组对应转换后的晶面
H1, K1, L1 = 0, 0, 8
H2, K2, L2 = 1, -3, 2
H3, K3, L3 = 1, 3, 2


h_list = [[h1, k1, l1 ], [h2, k2, l2], [h3, k3, l3]]      # 原晶面三组向量
H_list = [[H1, K1, L1 ], [H2, K2, L2], [H3, K3, L3]]      # 新晶面对应向量

S_plane = CrystalTransform.compute_S_from_planes(h_list, H_list)
print("S from planes =\n", S_plane)

S from planes =
 [[-0.5 -0.5  1. ]
 [ 1.5 -1.5  0. ]
 [ 3.   3.   2. ]]


In [300]:
# ===== 2. 已知三组晶向对应关系，求 S =====
# 输入三组原晶向
u1, v1, w1 = 1, 1, -2
u2, v2, w2 = 2, 1, -3
u3, v3, w3 = -1, -1, 1

# 输入三组对应转换后的晶向
U1, V1, W1 = -2, 0, 0
U2, V2, W2 = -3, 0.333, 0
U3, V3, W3 = 1.25, 0, -0.125


u_list = [[u1, v1, w1], [u2, v2, w2], [u3, v3, w3]]
U_list = [[U1, V1, W1], [U2, V2, W2], [U3, V3, W3]]

S_dir = CrystalTransform.compute_S_from_directions(u_list, U_list)
print("S from directions =\n", S_dir)

S from directions =
 [[-0.5       -0.5        1.       ]
 [ 1.5015015 -1.5015015  0.       ]
 [ 3.         3.         2.       ]]


### 2.3 三个坐标系之间的转化矩阵

In [14]:
#输入A——a转换矩阵：S_Aa
#输入A——a'转换矩阵：S_Aa_prime
S_Aa = [[-0.5, -0.5, 1],
        [0.5, -1, 0.5],
        [2.5, 2.5, 3]]

S_Aa_prime = [[-0.5, -0.5, 1],
              [1.5, -1.5, 0],
              [3, 3, 2]]

# 求解a——a'转换矩阵
S_aa_prime = CrystalTransform.compute_between_bases_1(S_Aa, S_Aa_prime)

print("a -> a' 转换矩阵 S_aa' =\n", S_aa_prime)

a -> a' 转换矩阵 S_aa' =
 [[ 1.  0.  0.]
 [-1.  2.  0.]
 [-1.  0.  1.]]


In [15]:
#输入A——a转换矩阵：S_Aa
#输入a——a'转换矩阵：S_aa_prime
S_Aa = np.array([[-0.5, -0.5, 1],
                [0.5, -1, 0.5],
                [2.5, 2.5, 3]])
S_aa_prime = np.array([[ 1,  0, 0],
                      [-1,  2,  0],
                      [-1,  0,  1]])

# 求解A——a'转换矩阵
S_Aa_prime = CrystalTransform.compute_between_bases_2(S_Aa, S_aa_prime)

print("A -> a' 转换矩阵 S_Aa' =\n", S_Aa_prime)

A -> a' 转换矩阵 S_Aa' =
 [[-0.5 -0.5  1. ]
 [ 1.5 -1.5  0. ]
 [ 3.   3.   2. ]]


## 3、六角晶系三四指数转换
- 3.1、晶向指数变换
- 3.2、晶面指数变换
- 3.3、等价晶向计算
- 3.4、等价晶面计算

In [40]:
import itertools
import numpy as np

class HexIndexConverter:

    # -------------------------
    # 基础方向转换
    # -------------------------
    @staticmethod
    def dir_3_to_4(U, V, W):
        u = (2 * U - V) / 3
        v = (2 * V - U) / 3
        t = -(U + V) / 3
        w = W
        return np.array([u, v, t, w], dtype=float)

    @staticmethod
    def dir_4_to_3(u, v, t, w):
        U = u - t
        V = v - t
        W = w
        return np.array([U, V, W], dtype=float)

    # -------------------------
    # 基础晶面转换
    # -------------------------
    @staticmethod
    def plane_3_to_4(h, k, l):
        i = -(h + k)
        return np.array([h, k, i, l], dtype=float)

    @staticmethod
    def plane_4_to_3(h, k, i, l):
        # 六方晶系条件 i = -(h + k)
        return np.array([h, k, l], dtype=float)


    # 辅助函数：整数化指数
    # -------------------------
    @staticmethod
    def to_integer_indices(indices, tol=1e-2, factor_range = 13):
        """
        将浮点指数转换为整数。
        
        参数：
            indices: list 或 ndarray，长度3或4，例如 [u,v,t] 或 [u,v,t,l]
            tol: 容差
        返回：
            numpy array 整数化指数
        """
        arr = np.array(indices, dtype=float)

        # 尝试找到合适倍数 factor，使浮点接近整数
        for factor in range(1, factor_range):
            candidate = arr * factor
            if np.all(np.abs(candidate - np.round(candidate)) < tol):
                int_arr = np.round(candidate).astype(int)
                gcd = np.gcd.reduce(int_arr)
                int_arr = int_arr // gcd
                return int_arr
    
        # 如果没有找到，直接四舍五入
        print('not found integer indices, please increase factor_range')
        return np.round(arr).astype(int)
        
    # -------------------------
    # 核心：生成等价四指数
    # -------------------------
    @staticmethod
    def generate_equivalent_4index(first_two, last_one, max_index):
        """
        first_two: [a, b]
        last_one: l 或 w
        max_index: 最大允许指数
        生成等价四指数 [u, v, t, l/w] 满足 t = -(u + v)
        """
        a, b = first_two
        abs_vals = [abs(a), abs(b)]
        # 生成前两位置换 + 符号
        perm_list = set(itertools.permutations(abs_vals))
        results = []

        for p in perm_list:
            for signs in itertools.product([-1, 1], repeat=2):
                u, v = p[0]*signs[0], p[1]*signs[1]
                t = -(u + v)  # 第三位强制满足规则
                # 判断是否超过最大指数
                if all(abs(x) <= max_index for x in [u, v, t, last_one]):
                    results.append(np.array([u, v, t, last_one], dtype=float))
        return results

    # -------------------------
    # 等价方向
    # -------------------------
    @staticmethod
    def equivalent_directions(U, V, W, max_index=3):
        u, v, t, w = HexIndexConverterFull.dir_3_to_4(U, V, W)
        eq4 = HexIndexConverterFull.generate_equivalent_4index([u, v], w, max_index)

        eq3 = []
        for a, b, c, d in eq4:
            U2, V2, W2 = HexIndexConverterFull.dir_4_to_3(a, b, c, d)
            eq3.append(tuple(np.round([U2, V2, W2]).astype(int)))

        eq3 = sorted(set(eq3), key=lambda x: (abs(x[0]) + abs(x[1]) + abs(x[2]), x))
        return eq3

    # -------------------------
    # 等价晶面
    # -------------------------
    @staticmethod
    def equivalent_planes(h, k, l, max_index=3):
        H, K, I, L = HexIndexConverterFull.plane_3_to_4(h, k, l)
        eq4 = HexIndexConverterFull.generate_equivalent_4index([H, K], L, max_index)

        eq3 = []
        for a, b, c, d in eq4:
            eq3.append(tuple(np.round([a, b, d]).astype(int)))

        eq3 = sorted(set(eq3), key=lambda x: (abs(x[0]) + abs(x[1]) + abs(x[2]), x))
        return eq3

convert = HexIndexConverter()

### 3.1、晶向指数变换

In [44]:
# 输入三指数晶向指数
U, V, W = -1, 0, 0

arr4 = convert.dir_3_to_4(U, V, W) 
print(convert.to_integer_indices(arr4))

[-2  1  1  0]


In [45]:
# 输入四指数晶向指数
u, v, t, w = -1, 0, 1, 0

arr3 = convert.dir_4_to_3(u, v, t, w)
print(convert.to_integer_indices(arr3))

[-2 -1  0]


### 3.2、晶面指数变换

In [46]:
# 输入三指数晶面指数
H, K, L = -2, -1, 0

arr4 = convert.plane_3_to_4(H, K, L)
print(convert.to_integer_indices(arr4))

[-2 -1  3  0]


In [47]:
# 输入四指数晶面指数
h, k, i, l = -1, 0, 1, 0

arr3 = convert.plane_4_to_3(h, k, i, l)
print(convert.to_integer_indices(arr3))

[-1  0  0]


### 3.3 等价晶向计算

In [17]:
# 输入三指数晶向指数
U, V, W = 1, 0, 0
# 允许最大的指数
max_index = 3

print(f'和晶向[{U} {V} {W}]等价的晶向：', conv.equivalent_directions(U, V, W, max_index))

和晶向[1 0 0]等价的晶向： [(-1, 0, 0), (0, -1, 0), (0, 1, 0), (1, 0, 0), (-2, -1, 0), (-1, -2, 0), (1, 2, 0), (2, 1, 0)]


### 3.4 等价晶面计算

In [16]:
# 输入三指数晶面指数
H, K, L = 1, 0, 0
# 允许最大的指数
max_index = 3

print(f'和晶面({H} {K} {L})等价的晶面：', conv.equivalent_planes(H, K, L, max_index))

和晶面(1 0 0)等价的晶面： [(-1, 0, 0), (0, -1, 0), (0, 1, 0), (1, 0, 0)]
